# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком.

Постройте модель с предельно большим значением F1-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте F1-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте AUC-ROC, сравнивайте её значение с F1-мерой.

## Откройте и изучите файл

In [302]:
import pandas as pd
from IPython.display import display
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [235]:
df = pd.read_csv('./datasets/Churn.csv')
display(df.head())
df.info()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


В данных 10000 объектов и 14 признаков (`Exited` - целевой признак). Каждый объект в наборе данных — это информация о поведении клиентов и расторжении договоров с банком. Известно:

- *RowNumber* — индекс строки в данных
- *CustomerId* — уникальный идентификатор клиента
- *Surname* — фамилия
- *CreditScore* — кредитный рейтинг
- *Geography* — страна проживания
- *Gender* — пол
- *Age* — возраст
- *Tenure* — сколько лет человек является клиентом банка
- *Balance* — баланс на счёте
- *NumOfProducts* — количество продуктов банка, используемых клиентом
- *HasCrCard* — наличие кредитной карты
- *IsActiveMember* — активность клиента
- *EstimatedSalary* — предполагаемая зарплата
- *Exited* — факт ухода клиента

В столбце `Tenure` 9% пропусков. В названиях всех колонок видно нарушение стиля. Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Подготовка данных

### Стиль заголовков
Переименуем столбцы:

In [236]:
df.columns = ['row_number', 'customer_id', 'surname', 'credit_score', 'geography', 'gender', 'age', 'tenure', 'balance', 'num_of_products', 'has_cr_card', 'is_active_member', 'estimated_salary', 'excited']

### Обработка пропусков

Скорее всего, пробелы в столбце `tenure` сигнализируют о том, что этот пользователь недавно стал клиентом банка. Поэтому заполним эти пропуска нулем:

In [237]:
df['tenure'] = df['tenure'].fillna(0)

### Бесполезные данные

В нашей таблице есть данные, не имеющие потенциальной связи с результатом работы. Эти данные не только полезны для модели, но могут навредить. Поэтому мы избавимся от таких столбцов, как `row_number`, `customer_id`, `surname`.

In [238]:
df = df.drop(['row_number', 'customer_id', 'surname'], axis=1)

### Прямое кодирование (OHE)

Наши данные содержат категориальные признаки `geography`, `gender`. Чтобы не возникло ошибки при обучении модели, преобразуем категориальные признаки в численные с помощью техники прямого кодирования (One-Hot Encoding, OHE), а чтобы не попасть в дамми-ловушку вызовом функции `pd.get_dummies()` с аргументом `drop_first`.

In [295]:
df = pd.get_dummies(df, drop_first=True)

Приведем все названия столбцов к нижнему регистру:

In [240]:
df.columns = df.columns.str.lower()

### Разбиение данных на выборки

Для того чтобы выполнять задачу классификации нам сначала необходимо разбить данные на три выборки: обучающую, валидационную и тестовую. Разобьем исходные данные в соотношении 3:1:1. Для начала используем метод `train_test_split`, чтобы отделить обучающую выборку от данных. После этого этим же методом разделим оставшиеся данные на две выборки - валидационную и тестовую.

In [241]:
df_train, df_test = train_test_split(df, test_size=0.4, random_state=12345)
df_test, df_valid = train_test_split(df_test, test_size=0.5, random_state=12345)

Для каждого набора данных выделим целевой признак (`target`) и другие признаки (`features`).

In [242]:
def get_features_and_target(data):
    return data.drop('excited', axis=1), data['excited']

features_train, target_train = get_features_and_target(df_train)

features_valid, target_valid = get_features_and_target(df_valid)

features_test, target_test = get_features_and_target(df_test)

### Масштабирование признаков

Чтобы избежать ловушки, когда алгоритм решит, что один признак важнее другого, признаки масштабируются — приводятся к одному масштабу. Стандартизируем признаки с помощью `StandardScaler`.


In [243]:
numeric = ['credit_score', 'age', 'tenure', 'balance', 'num_of_products', 'estimated_salary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

features_valid.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_germany,geography_spain,gender_male
7041,-2.226392,-0.088482,-0.825373,-1.233163,0.830152,1,0,0.647083,0,0,1
5709,-0.087120,0.006422,1.426375,-1.233163,-0.891560,1,0,-1.658410,0,0,0
7117,-0.917905,-0.752805,0.139662,0.722307,-0.891560,1,1,-1.369334,0,1,1
7775,-0.253277,0.101325,1.748053,-1.233163,0.830152,1,0,0.075086,0,1,1
8735,0.785204,-0.847708,1.748053,0.615625,-0.891560,0,1,-1.070919,0,0,1


### Баланс классов
Посмотрим, сбалансированы ли наши классы в данных

In [244]:
df['excited'].value_counts(normalize=True)

0    0.7963
1    0.2037
Name: excited, dtype: float64

В нашей задаче наблюдается сильный дисбаланс классов (4:1), что плохо скажется на обучении модели. Чтобы выровнять баланс, мы можем использовать такие техники, как: *взвешивание классов*, *upsampling* и *downsampling*. Но будем следовать очередности задач в проекте, и для начала научим модель на несбалансированных данных.

## Подготовка прототипа решения

Наш целевой признак – категоральный, это значит, что мы имеем дело с задачей классификации, а именно *бинарной классификации*, так как категорий всего две («клиет ушел» — `exited = 1`, «клиент остался» — `exited = 0`).

Для решения этой задачи нам подойдут следующие модели:
- дерево решений;
- случайный лес;
- логистическую регрессию.

Будем поочередно обучать три модели, а затем оценим их.

Сначала будем учить модель дерева решений. Для того чтобы получить максимально высокий уровень качества предсказаний, переберем в алгоритме обучения разные варианты глубины дерева от 1 до 30.

Так как в данных наблюдается дисбаланс классов, в качестве метрики для оценки качества всех моделей, мы будет использовать не *accuracy*, а *F1-score* (среднее гармоническое *precision* и *recall*).

In [250]:
random_state = 12345

def decision_tree(features_train, target_train, features_valid, target_valid, class_weight=None):
    best_model = None
    best_result = 0
    best_depth = 1

    for depth in range(1, 30, 1):
        model = DecisionTreeClassifier(random_state=random_state, max_depth=depth, class_weight=class_weight)
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        result = f1_score(target_valid, predicted_valid)
        if result > best_result:
            best_model = model
            best_depth = depth
            best_result = result
    return best_result, best_depth, best_model

decision_tree_result, decision_tree_depth, _ = decision_tree(
    features_train,
    target_train,
    features_valid,
    target_valid
)

print(f'F1 наилучшей модели дерева решений на валидационной выборке: {decision_tree_result:.4}. ',
      f'Глубина дерева: {decision_tree_depth}', sep='\n')

F1 наилучшей модели дерева решений на валидационной выборке: 0.5378. 
Глубина дерева: 9


Модель с лучшим значением *F1-score* (0.5378) на валидационной выборке оказалась модель с глубиной дерева 9. Это не очень хороший результат, но следует помнить, что мы не учли дисбаланс классов.

Попробуем натренировать модель случайного леса. Чтобы найти лучшую модель будем подбирать еще один гиперпараметр – количество деревьев (n_estimators) от 10 до 100 с шагом 10.

In [251]:
def random_forest(features_train, target_train, features_valid, target_valid, class_weight=None):
    best_model = None
    best_result = 0
    best_est = 10
    best_depth = 1

    for est in range(10, 100, 10):
        for depth in range(1, 30, 1):
            model = RandomForestClassifier(
                random_state=random_state,
                n_estimators=est,
                max_depth=depth,
                class_weight=class_weight
            )
            model.fit(features_train, target_train)
            predicted_valid = model.predict(features_valid)
            result = f1_score(target_valid, predicted_valid)
            if result > best_result:
                best_model = model
                best_result = result
                best_est = est
                best_depth = depth
    return best_result, best_est, best_depth, best_model

forest_result, forest_est, forest_depth, _ = random_forest(
    features_train,
    target_train,
    features_valid,
    target_valid
)

print(f'F1 наилучшей модели случайного леса на валидационной выборке: {forest_result:.4}',
      f'Количество деревьев: {forest_est}',
      f'Глубина дерева: {forest_depth}', sep='\n')

F1 наилучшей модели случайного леса на валидационной выборке: 0.5531
Количество деревьев: 50
Глубина дерева: 18


Моделью с лучшим значением *F1-score* (0.5531) на валидационной выборке оказалась модель с количеством деревьев 50, и глубиной - 18. Как видим, *F1-score* модели случайного леса немного выше чем модели дерева решений, но этого значение нам все равно недостаточно для приемлемого результата. Также можно выделить минус модели случайного дерева – скорость выполнения: чем больше древ, тем неторопливее работает модель.

Посмотрим какой результат *F1-score* даст модель логистической регрессии на несбалансированных классах.

In [252]:
def logistic_regression(features_train, target_train, features_valid, target_valid, class_weight=None):
    model = LogisticRegression(random_state=random_state, solver='liblinear', class_weight=class_weight)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    result = f1_score(target_valid,predicted_valid)
    return result, model

logistic_regression_result, _ = logistic_regression(
    features_train,
    target_train,
    features_valid,
    target_valid
)

print(f'F1 модели логистической регрессии на валидационной выборке: {logistic_regression_result:.4}')

F1 модели логистической регрессии на валидационной выборке: 0.2743


Среди трех моделей значение *F1-score* наименьшее у логистической регрессии - 0.2743.

Но давайте перейдем к балансировке классов, возможно на сбалансированных данных эта модель покажет лучший результат.

## Борьба с дисбалансом

Как мы уже решили раньше, чтобы выровнять баланс, мы можем использовать такие техники, как: *взвешивание классов*, *upsampling* и *downsampling*. Начнем работу с первого метода - *взвешивание классов*

### Взвешивание классов

Если указать в параметрах наших алгоритмов *class_weight='balanced'*, алгоритм посчитает, во сколько раз класс «0» встречается чаще класса «1». Обозначим это число N и новые классы будуть выглядят так:
- вес класса «0» = 1.0
- вес класса «1» = N

In [253]:
logistic_regression_result, _ = logistic_regression(
    features_train,
    target_train,
    features_valid,
    target_valid,
    'balanced'
)
print(f'F1 модели логистической регрессии на валидационной выборке: {logistic_regression_result:.4}')

F1 модели логистической регрессии на валидационной выборке: 0.4797


In [269]:
forest_result, forest_est, forest_depth, _ = random_forest(
    features_train,
    target_train,
    features_valid,
    target_valid,
    'balanced'
)

print(f'F1 наилучшей модели случайного леса на валидационной выборке: {forest_result:.4}',
      f'Количество деревьев: {forest_est}',
      f'Глубина дерева: {forest_depth}', sep='\n')

F1 наилучшей модели случайного леса на валидационной выборке: 0.6197
Количество деревьев: 70
Глубина дерева: 9


In [254]:
decision_tree_result, decision_tree_depth, _ = decision_tree(
    features_train,
    target_train,
    features_valid,
    target_valid,
    'balanced'
)

print(f'F1 наилучшей модели дерева решений на валидационной выборке: {decision_tree_result:.4}. ',
      f'Глубина дерева: {decision_tree_depth}', sep='\n')

F1 наилучшей модели дерева решений на валидационной выборке: 0.5809. 
Глубина дерева: 5


**Вывод**
Взвешивание классов помогло нам добиться неплохого результата. Среди трех алгоритмов можно выделить RandomForest, моделька с количеством деревьев 70 и глубиной дерева 9 дала нам *f1-score* - 0.6197

### Увеличение выборки (upsampling)
Редкий класс будем повторять несколько раз (в нашей случаи 4)

In [278]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)

    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

In [256]:
logistic_regression_result, _ = logistic_regression(
    features_upsampled,
    target_upsampled,
    features_valid,
    target_valid
)
print(f'F1 модели логистической регрессии на валидационной выборке: {logistic_regression_result:.4}')

F1 модели логистической регрессии на валидационной выборке: 0.4779


In [257]:
decision_tree_result, decision_tree_depth, _ = decision_tree(
    features_upsampled,
    target_upsampled,
    features_valid,
    target_valid
)

print(f'F1 наилучшей модели дерева решений на валидационной выборке: {decision_tree_result:.4}. ',
      f'Глубина дерева: {decision_tree_depth}', sep='\n')

F1 наилучшей модели дерева решений на валидационной выборке: 0.5809. 
Глубина дерева: 5


In [258]:
forest_result, forest_est, forest_depth, _ = random_forest(
    features_upsampled,
    target_upsampled,
    features_valid,
    target_valid
)

print(f'F1 наилучшей модели случайного леса на валидационной выборке: {forest_result:.4}',
      f'Количество деревьев: {forest_est}',
      f'Глубина дерева: {forest_depth}', sep='\n')

F1 наилучшей модели случайного леса на валидационной выборке: 0.6206
Количество деревьев: 30
Глубина дерева: 11


**Вывод**
Продублировав объекты малого класса 4 раза, мы сбалансировали классы. Это помогло достичь *F1-score* - 0.6206 (Модель случайного леса с количеством деревьев 30 и глубиной - 11).

### Уменьшение выборки (downsampling)
Вместо повторения редкого класса (1), уберём часть класса 0.

In [265]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=random_state)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=random_state)] + [target_ones])

    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=random_state)

    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

In [266]:
logistic_regression_result, _ = logistic_regression(
    features_downsampled,
    target_downsampled,
    features_valid,
    target_valid
)
print(f'F1 модели логистической регрессии на валидационной выборке: {logistic_regression_result:.4}')

F1 модели логистической регрессии на валидационной выборке: 0.4863


In [267]:
decision_tree_result, decision_tree_depth, _ = decision_tree(
    features_downsampled,
    target_downsampled,
    features_valid,
    target_valid
)

print(f'F1 наилучшей модели дерева решений на валидационной выборке: {decision_tree_result:.4}. ',
      f'Глубина дерева: {decision_tree_depth}', sep='\n')

F1 наилучшей модели дерева решений на валидационной выборке: 0.6074. 
Глубина дерева: 5


In [268]:
forest_result, forest_est, forest_depth, _ = random_forest(
    features_downsampled,
    target_downsampled,
    features_valid,
    target_valid
)

print(f'F1 наилучшей модели случайного леса на валидационной выборке: {forest_result:.4}',
      f'Количество деревьев: {forest_est}',
      f'Глубина дерева: {forest_depth}', sep='\n')

F1 наилучшей модели случайного леса на валидационной выборке: 0.5906
Количество деревьев: 10
Глубина дерева: 5


**Вывод**
При уменьшении выборки две модели дали результат *F1-score* выше 0.59:
- Модель дерева решений с глубиной дерева 5 - 0.6074
- Модель случайного леса с количеством деревьев 10 и глубиной 5 - 0.5906.

## Тестирование модели
Давайте выделим четыре модели из предыдущего задания, давшие результат *F1-score* на валидационной выборке выше 0.59 и сравним их результат на тестовой выборке.

Также выполним дополнительное задание и измерим значение *AUC-ROC* на тестовой выборки и сравним с *F1-score*.

По окончании выполнения этой задачи сделаем выводы и выберем наиболее подходящую модель для нашей задачи.

In [312]:
models = [
    {
        'name': 'Random Forest: class weighting',
        'model': RandomForestClassifier(
            random_state=random_state, n_estimators=70, max_depth=9, class_weight='balanced'),
        'features': features_train,
        'target': target_train,
        'f1_score_on_valid': 0.6197
    },
    {
        'name': 'Random Forest: upsampling',
        'model': RandomForestClassifier(random_state=random_state, n_estimators=30, max_depth=11),
        'features': features_upsampled,
        'target': target_upsampled,
        'f1_score_on_valid': 0.6206
    },
    {
        'name': 'Decision Tree: downsampling',
        'model': DecisionTreeClassifier(random_state=random_state, max_depth=5),
        'features': features_downsampled,
        'target': target_downsampled,
        'f1_score_on_valid': 0.6074
    },
    {
        'name': 'Random Forest: downsampling',
        'model': RandomForestClassifier(random_state=random_state, n_estimators=10, max_depth=5),
        'features': features_downsampled,
        'target': target_downsampled,
        'f1_score_on_valid': 0.5906
    },
]

for model_obj in models:
    model = model_obj['model']
    model.fit(model_obj['features'], model_obj['target'])

    #speed
    start = time.time()
    predicted_test = model.predict(features_test)
    end = time.time()
    speed = end - start

    #f1_score
    f1 =  f1_score(target_test, predicted_test)

    #auc_roc
    probabilities_test = model.predict_proba(features_test)
    probabilities_one_test = probabilities_test[:, 1]
    auc_roc = roc_auc_score(target_test, probabilities_one_test)

    model_obj['f1_score_on_test'] = round(f1, 4)
    model_obj['speed'] = round(speed, 5)
    model_obj['auc_roc_on_test'] = round(auc_roc, 4)

display(pd.DataFrame(models, columns=['name', 'f1_score_on_valid', 'f1_score_on_test', 'auc_roc_on_test', 'speed']))

,name,f1_score_on_valid,f1_score_on_test,auc_roc_on_test,speed
0,Random Forest: class weighting,0.6197,0.6224,0.8537,0.02424
1,Random Forest: upsampling,0.6206,0.6121,0.8434,0.01147
2,Decision Tree: downsampling,0.6074,0.5931,0.8229,0.00066
3,Random Forest: downsampling,0.5906,0.5890,0.8379,0.00204


### Вывод
